In [0]:
# https://keras.io/
!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, DepthwiseConv2D
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 10
num_filter = 20

In [3]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 263s 2us/step


In [0]:
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

In [0]:
"""
input = Input(shape=(img_height, img_width, channel,))

# Layer 1
layer1 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input)
layer1 = BatchNormalization(name='norm_1')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)
layer1 = MaxPooling2D(pool_size=(2, 2))(layer1)

# Layer 2
layer1 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_2')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)
layer1 = MaxPooling2D(pool_size=(2, 2))(layer1)

# Layer 3
layer1 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_3')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

# Layer 4
layer1 = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_4', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_4')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

# Layer 5
layer1 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_5', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_5')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)
layer1 = MaxPooling2D(pool_size=(2, 2))(layer1)

# Layer 6
layer1 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_6', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_6')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

# Layer 7
layer1 = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_7', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_7')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

# Layer 8
layer1 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_8', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_8')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)
layer1 = MaxPooling2D(pool_size=(2, 2))(layer1)

# Layer 9
layer1 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_9', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_9')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

# Layer 10
layer1 = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_10', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_10')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

# Layer 11
layer1 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_11', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_11')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

# Layer 12
layer1 = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_12', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_12')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

skip_connection = layer1

layer1 = MaxPooling2D(pool_size=(2, 2))(layer1)

# Layer 13
layer1 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_13', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_13')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)



# Layer 14
layer1 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_14', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_14')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

# Layer 15
layer1 = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_15', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_15')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

# Layer 16
layer1 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_16', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_16')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

# Layer 17
layer1 = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_17', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_17')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

# Layer 18
layer1 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_18', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_18')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

# Layer 19
layer1 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_19', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_19')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

# Layer 20
layer1 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_20', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_20')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

# Layer 21
skip_connection = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_21', use_bias=False)(skip_connection)
skip_connection = BatchNormalization(name='norm_21')(skip_connection)
skip_connection = LeakyReLU(alpha=0.1)(skip_connection)
skip_connection = Lambda(space_to_depth_x2)(skip_connection)

layer1 = concatenate([skip_connection, layer1])

# Layer 22
layer1 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_22', use_bias=False)(layer1)
layer1 = BatchNormalization(name='norm_22')(layer1)
layer1 = LeakyReLU(alpha=0.1)(layer1)

# Layer 23
layer1 = Flatten()(layer1)

output = Dense(num_classes, activation='softmax')(layer1)

"""

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
input = Input(shape=(img_height, img_width, channel,))

# Layer 1
layer1 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input)
layer1 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2', use_bias=False)(layer1)
layer1 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3', use_bias=False)(layer1)
layer1 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_4', use_bias=False)(layer1)
layer1 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_5', use_bias=False)(layer1)
layer1 = MaxPooling2D(pool_size=(2, 2))(layer1)
skip_connection = layer1

# Layer 2
layer1 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_6', use_bias=False)(layer1)
layer1 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_7', use_bias=False)(layer1)
layer1 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_8', use_bias=False)(layer1)
layer1 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_9', use_bias=False)(layer1)
layer1 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_10', use_bias=False)(layer1)
layer1 = MaxPooling2D(pool_size=(2, 2))(layer1)


# Layer 3
layer1 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_11', use_bias=False)(layer1)
layer1 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_12', use_bias=False)(layer1)
layer1 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_13', use_bias=False)(layer1)
layer1 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_14', use_bias=False)(layer1)
layer1 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_15', use_bias=False)(layer1)
layer1 = MaxPooling2D(pool_size=(2, 2))(layer1)


# Layer 4
skip_connection = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_16', use_bias=False)(skip_connection)
skip_connection = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_17', use_bias=False)(skip_connection)
skip_connection = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_18', use_bias=False)(skip_connection)
skip_connection = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_19', use_bias=False)(skip_connection)
skip_connection = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_20', use_bias=False)(skip_connection)
skip_connection = MaxPooling2D(pool_size=(2, 2))(skip_connection)



layer1 = concatenate([skip_connection, layer1])

layer1 = Flatten()(layer1)

output = Dense(num_classes, activation='softmax')(layer1)

ValueError: ignored

In [14]:
model = Model(inputs=[input], outputs=[output])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 32, 32, 32)        864       
_________________________________________________________________
conv_2 (Conv2D)              (None, 32, 32, 64)        18432     
_________________________________________________________________
conv_3 (Conv2D)              (None, 32, 32, 128)       73728     
_________________________________________________________________
conv_4 (Conv2D)              (None, 32, 32, 256)       294912    
_________________________________________________________________
conv_5 (Conv2D)              (None, 32, 32, 512)       1179648   
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 16, 16, 512)       0         
__________

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 106s 2ms/step - loss: 1.4343 - acc: 0.4787 - val_loss: 2.1679 - val_acc: 0.3629
Epoch 2/50
50000/50000 [==============================] - 92s 2ms/step - loss: 0.9772 - acc: 0.6498 - val_loss: 1.0646 - val_acc: 0.6414
Epoch 3/50
50000/50000 [==============================] - 92s 2ms/step - loss: 0.7734 - acc: 0.7295 - val_loss: 1.3574 - val_acc: 0.5768
Epoch 4/50
50000/50000 [==============================] - 92s 2ms/step - loss: 0.6515 - acc: 0.7735 - val_loss: 1.4387 - val_acc: 0.5554
Epoch 5/50
50000/50000 [==============================] - 92s 2ms/step - loss: 0.5543 - acc: 0.8079 - val_loss: 0.9330 - val_acc: 0.7149
Epoch 6/50
50000/50000 [==============================] - 92s 2ms/step - loss: 0.4665 - acc: 0.8388 - val_loss: 0.9897 - val_acc: 0.7071
Epoch 7/50
50000/50000 [==============================] - 92s 2ms/step - loss: 

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save the trained weights in to .h5 format
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")

10000/10000 [==============================] - 9s 936us/step
Test loss: 1.317255509376526
Test accuracy: 0.7693
Saved the model to disk


In [0]:
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")
from google.colab import files

files.download('Yolo_Basic_model.h5')

Saved the model to disk


FileNotFoundError: ignored